<a href="https://colab.research.google.com/github/acedesci/scanalytics/blob/master/S8_9_retail_analytics/S9_Module1B_Retail_Demand_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting demands for optimization models

In session 8, we designed our model on a predetermined subset of regressor variables and trained it by UPC. Now in this notebook, we will prepare our inputs for the optimization model by predicting the demands based on different price points.

In [0]:
import pandas
import sklearn
from sklearn import *

# 1. Data input

We have prepared the input files which contain the features to be predicted. The first file shows a small dataset whereas the second file consists of a large dataset, i.e.,

1.   **'InputFeatures_Prob1.csv'**. This is a small scale problem. The output of this will be used in the optimization model which you will see in Modules 2A (explicit model) and 2B (compact model).
2.   **'InputFeatures_Prob2.csv'**. This is a large-scale problem. This one contains a much higher number of variables and constraints to reflect real-life setting. We will use the output of this in the Module 2B.

In order to read the input, we provide two options here. Please run only either option 1 or option 2 (***not both***).

**Option 1: download from the URLs**. You can you can get it directly from the URLs as usual using the codes below to download 'InputFeatures_Prob1.csv' and save it in DataFrame


In [0]:
# small example
url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/S8_9_retail_analytics/InputFeatures_Prob1.csv'
# large example, please outcomment if you want to try
# url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/S8_9_retail_analytics/InputFeatures_Prob2.csv'

predDemand = pandas.read_csv(url)

# Dataset is now stored in a Pandas Dataframe predDemand
predDemand

,Unnamed: 0,avgPriceChoice,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE
0,0,3.0,1600027528,2.5,6.25,0,0,0,0.833333
1,1,3.0,1600027528,3.0,9.00,0,0,0,1.000000
2,2,3.0,1600027528,3.5,12.25,0,0,0,1.166667
3,3,3.0,1600027564,2.5,6.25,0,0,0,0.833333
4,4,3.0,1600027564,3.0,9.00,0,0,0,1.000000
5,5,3.0,1600027564,3.5,12.25,0,0,0,1.166667
6,6,3.0,3000006340,2.5,6.25,0,0,0,0.833333
7,7,3.0,3000006340,3.0,9.00,0,0,0,1.000000
8,8,3.0,3000006340,3.5,12.25,0,0,0,1.166667
9,9,3.0,3800031829,2.5,6.25,0,0,0,0.833333


**Option 2: Read the file directly from your drive.** You can upload the file from you PC. In this option, you must already download the file *'InputFeatures_Prob1.csv'* or 'InputFeatures_Prob2.csv' from Zonecours and save to your PC. Then you can upload it using the codes in the block below. After running the cell, click on "Choose Files" to upload it.

In [0]:
from google.colab import files
uploaded = files.upload()

import io
import pandas
predDemand = pandas.read_csv(io.BytesIO(uploaded['InputFeatures_Prob1.csv']))

# Dataset is now stored in a Pandas Dataframe predDemand
predDemand

Saving InputFeatures_Prob1.csv to InputFeatures_Prob1 (1).csv


,Unnamed: 0,avgPriceChoice,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE
0,0,3.0,1600027528,2.5,6.25,0,0,0,0.833333
1,1,3.0,1600027528,3.0,9.00,0,0,0,1.000000
2,2,3.0,1600027528,3.5,12.25,0,0,0,1.166667
3,3,3.0,1600027564,2.5,6.25,0,0,0,0.833333
4,4,3.0,1600027564,3.0,9.00,0,0,0,1.000000
5,5,3.0,1600027564,3.5,12.25,0,0,0,1.166667
6,6,3.0,3000006340,2.5,6.25,0,0,0,0.833333
7,7,3.0,3000006340,3.0,9.00,0,0,0,1.000000
8,8,3.0,3000006340,3.5,12.25,0,0,0,1.166667
9,9,3.0,3800031829,2.5,6.25,0,0,0,0.833333


# 2. Model retrieval

In the next two cells, we retrieve the best model that we previously trained and saved on Google Drive.

In [0]:
# we need to remount Google Drive in order to load the data from it
import pickle

from google.colab import drive
drive.mount('/content/drive')
cwd = '/content/drive/My Drive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We load the model that we previously trained and saved for each UPC.

In [0]:
productList = predDemand['UPC'].unique()

regr = {}
for upc in productList:
    filename = cwd+str(upc)+'_demand_model.sav'
    # save the model to disk
    regr[upc] = pickle.load(open(filename, 'rb'))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# 3. Demand forecasting

In this cell, we also create a loop **for** each UPC. Here are the descriptions of each line in the for loop

*   The first line in the **for** loop loads the data on the explanatory variables (features) for each UPC.
*   The second line retrives the UPC value so that we can call and run the model for that UPC.
*   The third line takes the model object for the current UPC (*regr[upc]*) and predicts the demand. We also use the function *clip(0.0)* to make sure that the demand is non-negative (which is possible since the demand is a decreasing function of price and the regression function is unbounded) and function *round(1)* to round the predicted value to one digit.
*   The fourth line put the predicted demand into the series which will be added as a new column
 
Once the for loop terminated, we add a new column *'predictSales'* which shows the predicted demand.

In [0]:
feature_list = ['PRICE', 'PRICE_p2', 'FEATURE', 'DISPLAY','TPR_ONLY','RELPRICE']

X = {}
y_pred = {}
predictedValueSeries = pandas.Series() # prepare blank series which will be added as a new column to the DataFrame predDemand

for upc in productList:
  X[upc] = predDemand.loc[predDemand['UPC']==upc][feature_list] # Line 1 of for loop: load the data on the explanatory variable
  upcIndex = predDemand.loc[predDemand['UPC']==upc].index # Line 2: retrieve the UPC value
  y_pred[upc] = regr[upc].predict(X[upc]).clip(0.0).round(1) # Line 3: predice the demands and make sure the demand is non-negative
  predictedValueSeries = predictedValueSeries.append(pandas.Series(y_pred[upc], index = upcIndex)) # Line 4: add the predicted demand to the series

predDemand['predictSales'] = predictedValueSeries
print(predDemand.to_string())

    Unnamed: 0  avgPriceChoice         UPC  PRICE  PRICE_p2  FEATURE  DISPLAY  TPR_ONLY  RELPRICE  predictSales
0            0             3.0  1600027528    2.5      6.25        0        0         0  0.833333          94.9
1            1             3.0  1600027528    3.0      9.00        0        0         0  1.000000          67.0
2            2             3.0  1600027528    3.5     12.25        0        0         0  1.166667          46.4
3            3             3.0  1600027564    2.5      6.25        0        0         0  0.833333          24.1
4            4             3.0  1600027564    3.0      9.00        0        0         0  1.000000          22.6
5            5             3.0  1600027564    3.5     12.25        0        0         0  1.166667          19.8
6            6             3.0  3000006340    2.5      6.25        0        0         0  0.833333           6.2
7            7             3.0  3000006340    3.0      9.00        0        0         0  1.000000       

Now we save the predicted sales into csv file to be used in the optimization model later on.

In [0]:
# Please save it as 'predictedSales_Prob1.csv' if 'InputFeatures_Prob1.csv' is used
# Otherwise, please save it as 'predictedSales_Prob2.csv' if 'InputFeatures_Prob2.csv' is used
predDemand.to_csv(cwd +'predictedSales_Prob1.csv') 